### Se tenía que utilizar tres fuentes (csv, api, rss, web scraping, Kaggle) de las cuales dos pueden ser del mismo formato y uno diferente, en esté caso utilize dos Web scraping y dos CSV. Se sacó información del número de casos de enfermos de Covid en cada uno de los estados de México y Brasil respectivamente. Tambíen se saco mapas de los países que presentan casos de Covid en el continente Americano y mapa de las capitales de Brasil que presentan casos.

### El dataframe resultante tiene el nombre de CovidMEX-BRA estados.csv

### El mapa resultante de Brasil es map_bra.html

### El mapa resultante del continente Americano es map_americano.html

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from pandas.io.json import json_normalize
import json
import folium
from folium import plugins
import matplotlib.pyplot as plt

## Wikipedia Coronavirus México

## Web scraping 1

In [336]:
url_over = 'https://es.wikipedia.org/wiki/Pandemia_de_COVID-19_en_México'

In [337]:
resp_over = requests.get(url_over, 'html.parser')

In [6]:
resp_over

<Response [200]>

In [338]:
over_sopa = bs(resp_over.content)

In [339]:
over_sopa.find

<bound method Tag.find of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Pandemia de COVID-19 en México - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"83729bbf-b3e9-4394-ae83-31e389c92331","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Pandemia_de_COVID-19_en_México","wgTitle":"Pandemia de COVID-19 en México","wgCurRevisionId":129886356,"wgRevisionId":129886356,"wgArticleId":9311970,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Páginas con plantillas con argumentos duplicados

In [340]:
table = over_sopa.find('table',{'class':'wikitable sortable col1izq col2der col3der col4der col5der'})
table

<table class="wikitable sortable col1izq col2der col3der col4der col5der" style="display: inline-table; font-size:90%; text-align:right;">
<caption>Casos de coronavirus en México por entidad federativa al 9 de octubre de 2020 <small>(<a href="/wiki/Tiempo_universal_coordinado" title="Tiempo universal coordinado">UTC</a>)</small>
</caption>
<tbody><tr>
<th rowspan="2" style="text-align:left;" width="140px">Estados
</th>
<th rowspan="2" width="60px">Casos
</th>
<th colspan="2">Fallecidos
</th>
<th colspan="2">Recuperado(s)
</th>
<th rowspan="2" width="60px">Casos por 100,000 de habitantes
</th>
<th rowspan="2" width="30px">Reporte del gobierno local
</th></tr>
<tr>
<th width="40px"><abbr title="Cantidad">Cant.</abbr>
</th>
<th width="40px">%
</th>
<th width="40px"><abbr title="Cantidad">Cant.</abbr>
</th>
<th width="40px">%
</th></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/Pandemia_de_enfermedad_por_coronavirus_de_2020_en_la_Ciudad_de_M%C3%A9xico" title="Pandemia de enfermedad por c

In [341]:
cabecera = table.findAll('th')

In [342]:
cabecera

[<th rowspan="2" style="text-align:left;" width="140px">Estados
 </th>,
 <th rowspan="2" width="60px">Casos
 </th>,
 <th colspan="2">Fallecidos
 </th>,
 <th colspan="2">Recuperado(s)
 </th>,
 <th rowspan="2" width="60px">Casos por 100,000 de habitantes
 </th>,
 <th rowspan="2" width="30px">Reporte del gobierno local
 </th>,
 <th width="40px"><abbr title="Cantidad">Cant.</abbr>
 </th>,
 <th width="40px">%
 </th>,
 <th width="40px"><abbr title="Cantidad">Cant.</abbr>
 </th>,
 <th width="40px">%
 </th>,
 <th>Totales</th>,
 <th>710 949</th>,
 <th>74 604</th>,
 <th>11.81</th>,
 <th>469 206</th>,
 <th>56.15</th>,
 <th>125.2</th>,
 <th><sup class="reference separada" id="cite_ref-132"><a href="#cite_note-132"><span class="corchete-llamada">[</span>132<span class="corchete-llamada">]</span></a></sup>​
 </th>]

In [343]:
cabecera[0].text

'Estados\n'

In [344]:
cabe_txt = [i.text.replace('\n','') for i in cabecera]

In [345]:
print(cabe_txt)

['Estados', 'Casos', 'Fallecidos', 'Recuperado(s)', 'Casos por 100,000 de habitantes', 'Reporte del gobierno local', 'Cant.', '%', 'Cant.', '%', 'Totales', '710\xa0949', '74\xa0604', '11.81', '469\xa0206', '56.15', '125.2', '[132]\u200b']


In [346]:
cabe_txt.pop(8), cabe_txt.pop(9), cabe_txt.pop(10), cabe_txt.pop(11), cabe_txt.pop(12)

('Cant.', 'Totales', '74\xa0604', '469\xa0206', '125.2')

In [347]:
cabe_txt.pop(8), cabe_txt.pop(9), cabe_txt.pop(10)

('%', '11.81', '[132]\u200b')

In [348]:
cabe_txt.pop(8)

'710\xa0949'

In [349]:
cabe_txt.pop(8)

'56.15'

In [350]:
print(cabe_txt)

['Estados', 'Casos', 'Fallecidos', 'Recuperado(s)', 'Casos por 100,000 de habitantes', 'Reporte del gobierno local', 'Cant.', '%']


In [351]:
body = table.find('tbody')

In [352]:
body_tr = body.findAll('tr')

In [353]:
body_td = body.findAll('td')

In [354]:
body_td[1].text

'120\xa0810'

In [355]:
body_td_txt = [str(i.text.replace('\n','')) for i in body_td]

In [29]:
#body_td_txt = [str(i.text.replace(' ','')) for i in body_td]

In [356]:
body_td_txt[:8]

['Ciudad de México',
 '120\xa0810',
 '12099',
 '10.11',
 '57\xa0562',
 '51.95',
 '1242.5',
 '[100]\u200b']

In [357]:
j = 0
lst_lst = []
for i in range(0,len(body_td_txt),8):

    lst_lst.append(body_td_txt[j:i])
    j = i

In [358]:
lst_lst

[[],
 ['Ciudad de México',
  '120\xa0810',
  '12099',
  '10.11',
  '57\xa0562',
  '51.95',
  '1242.5',
  '[100]\u200b'],
 ['Estado de México',
  '78\xa0353',
  '9170',
  '12.5',
  '39038',
  '44.51',
  '206.8',
  '[101]\u200b'],
 ['Nuevo León',
  '61720',
  '3386',
  '3.32',
  '15602',
  '63.04',
  '100.8',
  '[102]\u200b'],
 ['Jalisco',
  '64778',
  '2803',
  '5.12',
  '10198',
  '22.54',
  '188.3',
  '[103]\u200b'],
 ['Guanajuato',
  '39349',
  '2802',
  '6.14',
  '17637',
  '41.59',
  '175.1',
  '[104]\u200b'],
 ['Veracruz',
  '30790',
  '3938',
  '15.28',
  '16410',
  '47.94',
  '116.3',
  '[105]\u200b'],
 ['Tabasco',
  '29916',
  '2699',
  '9.71',
  '17380',
  '78.67',
  '526.6',
  '[106]\u200b'],
 ['Puebla',
  '29047',
  '3805',
  '12.23',
  '15534',
  '40.96',
  '183.2',
  '[107]\u200b'],
 ['Sonora', '25006', '2804', '11.21', '13382', '5.33', '369.1', '[108]\u200b'],
 ['Tamaulipas',
  '26780',
  '1963',
  '6.54',
  '14630',
  '44.52',
  '237.1',
  '[109]\u200b'],
 ['Coahuila',
 

In [359]:
lst_lst.pop(0)

[]

In [360]:
lst_lst

[['Ciudad de México',
  '120\xa0810',
  '12099',
  '10.11',
  '57\xa0562',
  '51.95',
  '1242.5',
  '[100]\u200b'],
 ['Estado de México',
  '78\xa0353',
  '9170',
  '12.5',
  '39038',
  '44.51',
  '206.8',
  '[101]\u200b'],
 ['Nuevo León',
  '61720',
  '3386',
  '3.32',
  '15602',
  '63.04',
  '100.8',
  '[102]\u200b'],
 ['Jalisco',
  '64778',
  '2803',
  '5.12',
  '10198',
  '22.54',
  '188.3',
  '[103]\u200b'],
 ['Guanajuato',
  '39349',
  '2802',
  '6.14',
  '17637',
  '41.59',
  '175.1',
  '[104]\u200b'],
 ['Veracruz',
  '30790',
  '3938',
  '15.28',
  '16410',
  '47.94',
  '116.3',
  '[105]\u200b'],
 ['Tabasco',
  '29916',
  '2699',
  '9.71',
  '17380',
  '78.67',
  '526.6',
  '[106]\u200b'],
 ['Puebla',
  '29047',
  '3805',
  '12.23',
  '15534',
  '40.96',
  '183.2',
  '[107]\u200b'],
 ['Sonora', '25006', '2804', '11.21', '13382', '5.33', '369.1', '[108]\u200b'],
 ['Tamaulipas',
  '26780',
  '1963',
  '6.54',
  '14630',
  '44.52',
  '237.1',
  '[109]\u200b'],
 ['Coahuila',
  '239

In [361]:
tabla_coronavirus_mexico = pd.DataFrame(lst_lst, columns = cabe_txt)

In [362]:
tabla_coronavirus_mexico.columns = ['Estados', 'Casos', 'Fallecidos', 'Fallecidos %', 'Recuperados',
       'Recuperados %', 'Casos por 100,000 de habitantes', 'Reporte del gobierno local']

In [363]:
tabla_coronavirus_mexico.head()

,Estados,Casos,Fallecidos,Fallecidos %,Recuperados,Recuperados %,"Casos por 100,000 de habitantes",Reporte del gobierno local
0,Ciudad de México,120 810,12099,10.11,57 562,51.95,1242.5,[100]​
1,Estado de México,78 353,9170,12.5,39038,44.51,206.8,[101]​
2,Nuevo León,61720,3386,3.32,15602,63.04,100.8,[102]​
3,Jalisco,64778,2803,5.12,10198,22.54,188.3,[103]​
4,Guanajuato,39349,2802,6.14,17637,41.59,175.1,[104]​


In [364]:
tabla_coronavirus_mexico.shape

(32, 8)

In [389]:
covid_mexico = tabla_coronavirus_mexico[['Estados', 'Casos', 'Fallecidos']]

In [390]:
covid_mexico.head()

,Estados,Casos,Fallecidos
0,Ciudad de México,120 810,12099
1,Estado de México,78 353,9170
2,Nuevo León,61720,3386
3,Jalisco,64778,2803
4,Guanajuato,39349,2802


## Wikipedia Coronavirus Brasil

## Web scraping 2

In [2]:
url_bra = 'https://es.wikipedia.org/wiki/Pandemia_de_COVID-19_en_Brasil'

In [3]:
resp_bra = requests.get(url_bra, 'html.parser')
resp_bra

<Response [200]>

In [4]:
sopa_bra = bs(resp_bra.content)

In [5]:
sopa_bra.find

<bound method Tag.find of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Pandemia de COVID-19 en Brasil - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"f0e30445-8db8-49f2-be13-363fff37d490","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Pandemia_de_COVID-19_en_Brasil","wgTitle":"Pandemia de COVID-19 en Brasil","wgCurRevisionId":129704870,"wgRevisionId":129704870,"wgArticleId":9321563,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Páginas con referencias con parámetros sugeridos

In [6]:
table_bra = sopa_bra.find('table',{'class':'wikitable sortable col3der'})
table_bra

<table class="wikitable sortable col3der">
<tbody><tr>
<th>Estado
</th>
<th>Casos confirmados
</th>
<th>Muertes confirmadas
</th>
<th>Letalidad
</th></tr>
<tr>
<td><a class="image" href="/wiki/Archivo:Bandeira_do_estado_de_S%C3%A3o_Paulo.svg"><img alt="Bandeira do estado de São Paulo.svg" class="thumbborder" data-file-height="520" data-file-width="780" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Bandeira_do_estado_de_S%C3%A3o_Paulo.svg/20px-Bandeira_do_estado_de_S%C3%A3o_Paulo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Bandeira_do_estado_de_S%C3%A3o_Paulo.svg/30px-Bandeira_do_estado_de_S%C3%A3o_Paulo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Bandeira_do_estado_de_S%C3%A3o_Paulo.svg/40px-Bandeira_do_estado_de_S%C3%A3o_Paulo.svg.png 2x" width="20"/></a> <a class="mw-redirect" href="/wiki/Pandemia_de_enfermedad_por_coronavirus_de_2020_en_S%C3%A3o_Paulo" title="Pandemia de enfermedad por coronavirus

In [8]:
cabecera_bra = table_bra.findAll('th')

In [9]:
cabecera_bra

[<th>Estado
 </th>,
 <th>Casos confirmados
 </th>,
 <th>Muertes confirmadas
 </th>,
 <th>Letalidad
 </th>,
 <th><a href="/wiki/Brasil" title="Brasil">Brasil</a>
 </th>]

In [10]:
cabecera_bra[0].text

'Estado\n'

In [11]:
cabe_txt_bra = [i.text.replace('\n','') for i in cabecera_bra]

In [12]:
cabe_txt_bra

['Estado', 'Casos confirmados', 'Muertes confirmadas', 'Letalidad', 'Brasil']

In [13]:
cabe_txt_bra.pop(4)

'Brasil'

In [14]:
body_bra = table_bra.find('tbody')

In [15]:
body_tr_bra = body_bra.findAll('tr')

In [16]:
body_td_bra = body_bra.findAll('td')

In [17]:
body_td_bra[0].text

' São Paulo\n'

In [18]:
body_td_txt_bra = [str(i.text.replace('\n','')) for i in body_td_bra]

In [19]:
body_td_txt_bra[:4]

[' São Paulo', '109698', '7615', '6.9%']

In [20]:
body_td_txt_bra

[' São Paulo',
 '109698',
 '7615',
 '6.9%',
 ' Minas Gerais',
 '10464',
 '271',
 '2.6%',
 ' Río de Janeiro',
 '53388',
 '5344',
 '10.1%',
 ' Bahía',
 '18392',
 '667',
 '3.6%',
 ' Río Grande del Sur',
 '9332',
 '224',
 '2.4%',
 ' Paraná',
 '4687',
 '182',
 '3.8%',
 ' Pernambuco',
 '34450',
 '2807',
 '8.1%',
 ' Ceará',
 '48489',
 '3010',
 '6.2%',
 'Pará',
 '37961',
 '2923',
 '7.6%',
 ' Goiás',
 '3702',
 '124',
 '3.3%',
 ' Maranhão',
 '34639',
 '955',
 '2.7%',
 ' Santa Catarina',
 '9037',
 '143',
 '1.6%',
 ' Paraíba',
 '13162',
 '360',
 '2.7%',
 ' Amazonas',
 '41378',
 '2052',
 '4.9%',
 ' Espírito Santo',
 '13690',
 '604',
 '4.4%',
 ' Río Grande del Norte',
 '8051',
 '305',
 '3.7%',
 ' Alagoas',
 '10288',
 '443',
 '4.3%',
 ' Mato Grosso',
 '2429',
 '56',
 '2.5%',
 ' Piauí',
 '4931',
 '168',
 '3.3%',
 ' Distrito Federal',
 '9780',
 '170',
 '1.7%',
 ' Mato Grosso del Sur',
 '1489',
 '20',
 '1.3%',
 ' Sergipe',
 '6999',
 '158',
 '2.5%',
 ' Rondonia',
 '4942',
 '156',
 '3.2%',
 ' Tocantins',


In [22]:
j = 0
lst_lst_bra = []
for i in range(0,len(body_td_txt_bra),4):

    lst_lst_bra.append(body_td_txt_bra[j:i])
    j = i

In [26]:
lst_lst_bra.pop(0)

[]

In [27]:
lst_lst_bra

[[' São Paulo', '109698', '7615', '6.9%'],
 [' Minas Gerais', '10464', '271', '2.6%'],
 [' Río de Janeiro', '53388', '5344', '10.1%'],
 [' Bahía', '18392', '667', '3.6%'],
 [' Río Grande del Sur', '9332', '224', '2.4%'],
 [' Paraná', '4687', '182', '3.8%'],
 [' Pernambuco', '34450', '2807', '8.1%'],
 [' Ceará', '48489', '3010', '6.2%'],
 ['Pará', '37961', '2923', '7.6%'],
 [' Goiás', '3702', '124', '3.3%'],
 [' Maranhão', '34639', '955', '2.7%'],
 [' Santa Catarina', '9037', '143', '1.6%'],
 [' Paraíba', '13162', '360', '2.7%'],
 [' Amazonas', '41378', '2052', '4.9%'],
 [' Espírito Santo', '13690', '604', '4.4%'],
 [' Río Grande del Norte', '8051', '305', '3.7%'],
 [' Alagoas', '10288', '443', '4.3%'],
 [' Mato Grosso', '2429', '56', '2.5%'],
 [' Piauí', '4931', '168', '3.3%'],
 [' Distrito Federal', '9780', '170', '1.7%'],
 [' Mato Grosso del Sur', '1489', '20', '1.3%'],
 [' Sergipe', '6999', '158', '2.5%'],
 [' Rondonia', '4942', '156', '3.2%'],
 [' Tocantins', '4176', '73', '1.7%'],

In [28]:
tabla_coronavirus_brasil = pd.DataFrame(lst_lst_bra, columns = cabe_txt_bra)

,Estado,Casos confirmados,Muertes confirmadas,Letalidad
0,São Paulo,109698,7615,6.9%
1,Minas Gerais,10464,271,2.6%
2,Río de Janeiro,53388,5344,10.1%
3,Bahía,18392,667,3.6%
4,Río Grande del Sur,9332,224,2.4%


In [385]:
tabla_coronavirus_brasil.columns = ['Estados', 'Casos', 'Fallecidos', 'Letalidad']

In [386]:
tabla_coronavirus_brasil.head()

,Estados,Casos,Fallecidos,Letalidad
0,São Paulo,109698,7615,6.9%
1,Minas Gerais,10464,271,2.6%
2,Río de Janeiro,53388,5344,10.1%
3,Bahía,18392,667,3.6%
4,Río Grande del Sur,9332,224,2.4%


In [379]:
tabla_coronavirus_brasil.shape

(27, 4)

In [387]:
covid_brasil = tabla_coronavirus_brasil[['Estados', 'Casos', 'Fallecidos']]

In [388]:
covid_brasil.head()

,Estados,Casos,Fallecidos
0,São Paulo,109698,7615
1,Minas Gerais,10464,271
2,Río de Janeiro,53388,5344
3,Bahía,18392,667
4,Río Grande del Sur,9332,224


## Usando CSV 

## Para cordenadas de capitales de Brasil

In [212]:
csv_bra = pd.read_csv('/Users/martin/desktop/csv/brazil_cities_coordinates.csv')

In [213]:
csv_bra.head()

,state_code,city_code,city_name,lat,long,capital
0,52,5200050,Abadia de Goiás,-16.75730,-49.4412,False
1,31,3100104,Abadia dos Dourados,-18.48310,-47.3916,False
2,52,5200100,Abadiânia,-16.19700,-48.7057,False
3,31,3100203,Abaeté,-19.15510,-45.4444,False
4,15,1500107,Abaetetuba,-1.72183,-48.8788,False


In [365]:
csv_bra_new = csv_bra[csv_bra['capital'] == True]

In [367]:
csv_bra_new = csv_bra_new.reset_index()

In [369]:
csv_bra_new.drop(['index'], axis = 1, inplace = True)

In [370]:
csv_bra_new.head()

,state_code,city_code,city_name,lat,long,capital
0,28,2800308,Aracaju,-10.90910,-37.0677,True
1,15,1501402,Belém,-1.45540,-48.4898,True
2,31,3106200,Belo Horizonte,-19.91020,-43.9266,True
3,14,1400100,Boa Vista,2.82384,-60.6753,True
4,53,5300108,Brasília,-15.77950,-47.9297,True


In [374]:
mapa_bra = csv_bra_new[['lat', 'long']]
mapa_bra

,lat,long
0,-10.909100,-37.0677
1,-1.455400,-48.4898
2,-19.910200,-43.9266
3,2.823840,-60.6753
4,-15.779500,-47.9297
5,-20.448600,-54.6295
6,-15.601000,-56.0974
7,-25.419500,-49.2646
8,-27.594500,-48.5477
9,-3.716640,-38.5423


## Usando CSV covid mundial

In [214]:
csv_mun = pd.read_csv('/Users/martin/desktop/csv/covid_19_clean_complete.csv')

In [237]:
csv_mun.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [238]:
csv_mun.shape

(49068, 10)

In [247]:
csv_mun['Date'].unique()

array(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
       '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02',
       '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10',
       '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18',
       '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22',
       '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26',
       '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01',
       '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
       '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
       '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
       '2020-03-22', '2020-03-23', '2020-03-24', '2020-

In [252]:
csv_df = csv_mun[csv_mun['Date']== '2020-07-27']

In [253]:
csv_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
48807,NaN,Afghanistan,33.93911,67.709953,2020-07-27,36263,1269,25198,9796,Eastern Mediterranean
48808,NaN,Albania,41.15330,20.168300,2020-07-27,4880,144,2745,1991,Europe
48809,NaN,Algeria,28.03390,1.659600,2020-07-27,27973,1163,18837,7973,Africa
48810,NaN,Andorra,42.50630,1.521800,2020-07-27,907,52,803,52,Europe
48811,NaN,Angola,-11.20270,17.873900,2020-07-27,950,41,242,667,Africa


In [259]:
csv_df = csv_df.reset_index()
csv_df.head()

,index,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,48807,NaN,Afghanistan,33.93911,67.709953,2020-07-27,36263,1269,25198,9796,Eastern Mediterranean
1,48808,NaN,Albania,41.15330,20.168300,2020-07-27,4880,144,2745,1991,Europe
2,48809,NaN,Algeria,28.03390,1.659600,2020-07-27,27973,1163,18837,7973,Africa
3,48810,NaN,Andorra,42.50630,1.521800,2020-07-27,907,52,803,52,Europe
4,48811,NaN,Angola,-11.20270,17.873900,2020-07-27,950,41,242,667,Africa


In [254]:
csv_df.shape

(261, 10)

In [262]:
csv_df['WHO Region'].unique()

array(['Eastern Mediterranean', 'Europe', 'Africa', 'Americas',
       'Western Pacific', 'South-East Asia'], dtype=object)

In [264]:
csv_df.drop(['index', 'Province/State', 'Date', 'Active'], axis = 1, inplace = True)

In [270]:
csv_df.rename(columns = {'Country/Region':'Country'}, inplace = True)

In [271]:
csv_df.head()

,Country,Lat,Long,Confirmed,Deaths,Recovered,WHO Region
0,Afghanistan,33.93911,67.709953,36263,1269,25198,Eastern Mediterranean
1,Albania,41.15330,20.168300,4880,144,2745,Europe
2,Algeria,28.03390,1.659600,27973,1163,18837,Africa
3,Andorra,42.50630,1.521800,907,52,803,Europe
4,Angola,-11.20270,17.873900,950,41,242,Africa


In [301]:
df = csv_df[csv_df['WHO Region'] == 'Americas']

In [302]:
df = df.reset_index()

In [303]:
df.drop(['index'], axis = 1, inplace = True)

In [304]:
df.head()

,Country,Lat,Long,Confirmed,Deaths,Recovered,WHO Region
0,Antigua and Barbuda,17.060800,-61.796400,86,3,65,Americas
1,Argentina,-38.416100,-63.616700,167416,3059,72575,Americas
2,Bahamas,25.025885,-78.035889,382,11,91,Americas
3,Barbados,13.193900,-59.543200,110,7,94,Americas
4,Bolivia,-16.290200,-63.588700,71181,2647,21478,Americas


In [305]:
df

,Country,Lat,Long,Confirmed,Deaths,Recovered,WHO Region
0,Antigua and Barbuda,17.060800,-61.796400,86,3,65,Americas
1,Argentina,-38.416100,-63.616700,167416,3059,72575,Americas
2,Bahamas,25.025885,-78.035889,382,11,91,Americas
3,Barbados,13.193900,-59.543200,110,7,94,Americas
4,Bolivia,-16.290200,-63.588700,71181,2647,21478,Americas
5,Brazil,-14.235000,-51.925300,2442375,87618,1846641,Americas
6,Canada,53.933300,-116.576500,10390,186,0,Americas
7,Canada,53.726700,-127.647600,3500,193,0,Americas
8,Canada,53.760900,-98.813900,400,7,0,Americas
9,Canada,46.565300,-66.461900,170,2,0,Americas


In [306]:
canada_df = df[df['Country'] == 'Canada']

In [307]:
canada_df['Confirmed'].sum()

116458

In [308]:
canada_df['Deaths'].sum()

8944

In [309]:
canada_df_sum = pd.DataFrame({'Country':['Canada'], 'Lat':[53.7609], 'Long':[-127.6476], 'Confirmed':[116458], 'Deaths':[8944], 'Recovered':[0], 'WHO Region':['Americas']})

In [310]:
canada_df_sum

,Country,Lat,Long,Confirmed,Deaths,Recovered,WHO Region
0,Canada,53.7609,-127.6476,116458,8944,0,Americas


In [319]:
df = df[df['Country'] != 'Canada']

In [320]:
df.head()

,Country,Lat,Long,Confirmed,Deaths,Recovered,WHO Region
0,Antigua and Barbuda,17.060800,-61.796400,86,3,65,Americas
1,Argentina,-38.416100,-63.616700,167416,3059,72575,Americas
2,Bahamas,25.025885,-78.035889,382,11,91,Americas
3,Barbados,13.193900,-59.543200,110,7,94,Americas
42,Belize,17.189900,-88.497600,48,2,26,Americas
4,Bolivia,-16.290200,-63.588700,71181,2647,21478,Americas
5,Brazil,-14.235000,-51.925300,2442375,87618,1846641,Americas
16,Chile,-35.675100,-71.543000,347923,9187,319954,Americas
17,Colombia,4.570900,-74.297300,257101,8777,131161,Americas
18,Costa Rica,9.748900,-83.753400,15841,115,3824,Americas


In [328]:
df = df.append(canada_df_sum)

In [329]:
df = df.sort_values('Country', ascending = True)

In [330]:
df = df.reset_index()

In [332]:
df.drop(['index'], axis = 1, inplace = True)

In [334]:
df.head()

,Country,Lat,Long,Confirmed,Deaths,Recovered,WHO Region
0,Antigua and Barbuda,17.060800,-61.796400,86,3,65,Americas
1,Argentina,-38.416100,-63.616700,167416,3059,72575,Americas
2,Bahamas,25.025885,-78.035889,382,11,91,Americas
3,Barbados,13.193900,-59.543200,110,7,94,Americas
4,Belize,17.189900,-88.497600,48,2,26,Americas


## Uniendo dataframes

In [406]:
df_mex_bra = pd.concat([covid_mexico, covid_brasil], axis = 0)

In [407]:
df_mex_bra = df_mex_bra.reset_index()

In [408]:
df_mex_bra.drop(['index'], axis = 1, inplace = True)

In [409]:
df_mex_bra.head()

,Estados,Casos,Fallecidos
0,Ciudad de México,120 810,12099
1,Estado de México,78 353,9170
2,Nuevo León,61720,3386
3,Jalisco,64778,2803
4,Guanajuato,39349,2802


In [412]:
df_mex_bra.shape

(59, 3)

In [411]:
df_mex_bra.to_csv("CovidMEX-BRA estados.csv")

## Haciendo el mapa de Brasil

In [418]:
mapa = folium.Map([-11.108985, -48.920686], zoom_start=4)

data = mapa_bra[['lat','long']].values

In [415]:
data

array([[-1.09091e+01, -3.70677e+01],
       [-1.45540e+00, -4.84898e+01],
       [-1.99102e+01, -4.39266e+01],
       [ 2.82384e+00, -6.06753e+01],
       [-1.57795e+01, -4.79297e+01],
       [-2.04486e+01, -5.46295e+01],
       [-1.56010e+01, -5.60974e+01],
       [-2.54195e+01, -4.92646e+01],
       [-2.75945e+01, -4.85477e+01],
       [-3.71664e+00, -3.85423e+01],
       [-1.66864e+01, -4.92643e+01],
       [-7.11509e+00, -3.48641e+01],
       [ 3.49340e-02, -5.10694e+01],
       [-9.66599e+00, -3.57350e+01],
       [-3.11866e+00, -6.00212e+01],
       [-5.79357e+00, -3.51986e+01],
       [-1.02400e+01, -4.83558e+01],
       [-3.00318e+01, -5.12065e+01],
       [-8.76077e+00, -6.38999e+01],
       [-8.04666e+00, -3.48771e+01],
       [-9.97499e+00, -6.78243e+01],
       [-2.29129e+01, -4.32003e+01],
       [-1.29718e+01, -3.85011e+01],
       [-2.53874e+00, -4.42825e+01],
       [-2.35329e+01, -4.66395e+01],
       [-5.09194e+00, -4.28034e+01],
       [-2.03155e+01, -4.03128e+01]])

In [420]:
mapa.add_child(plugins.HeatMap(data, radius = 15))
mapa

In [421]:
mapa.save('map_bra.html')

## Haciendo mapa del continente Americano

In [445]:
mapa2 = folium.Map([8.945512, -79.559003], zoom_start = 3)

data2 = df[['Lat','Long']].values

In [446]:
data2

array([[  17.0608  ,  -61.7964  ],
       [ -38.4161  ,  -63.6167  ],
       [  25.025885,  -78.035889],
       [  13.1939  ,  -59.5432  ],
       [  17.1899  ,  -88.4976  ],
       [ -16.2902  ,  -63.5887  ],
       [ -14.235   ,  -51.9253  ],
       [  53.7609  , -127.6476  ],
       [ -35.6751  ,  -71.543   ],
       [   4.5709  ,  -74.2973  ],
       [   9.7489  ,  -83.7534  ],
       [  21.521757,  -77.781167],
       [  15.415   ,  -61.371   ],
       [  18.7357  ,  -70.1627  ],
       [  -1.8312  ,  -78.1834  ],
       [  13.7942  ,  -88.8965  ],
       [  12.1165  ,  -61.679   ],
       [  15.7835  ,  -90.2308  ],
       [   4.860416,  -58.93018 ],
       [  18.9712  ,  -72.2852  ],
       [  15.2     ,  -86.2419  ],
       [  18.1096  ,  -77.2975  ],
       [  23.6345  , -102.5528  ],
       [  12.865416,  -85.207229],
       [   8.538   ,  -80.7821  ],
       [ -23.4425  ,  -58.4438  ],
       [  -9.19    ,  -75.0152  ],
       [  17.357822,  -62.782998],
       [  13.9094  ,

In [447]:
mapa2.add_child(plugins.HeatMap(data2, radius = 15))
mapa2

In [448]:
mapa2.save('map_americano.html')